In [2]:
print(" NS LONI ")

 NS LONI 


In [2]:
pip uninstall tensorflow


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)
Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl (210 kB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [5]:
import tensorflow as tf


🔹 1. Tensors and Tensor Operations


In [ ]:
# tf.constant ==> im-mutable ==>  creates fixed tensors.

const_tensor = tf.constant([[1, 2, 3], [4, 5, 6]])
print("constant = \n", const_tensor)

constant = 
 tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


In [7]:
# tf.Variable: mutable tensor
var_tensor = tf.Variable([[5, 6], [7, 8]])
print("Variable Tensor:\n", var_tensor)

Variable Tensor:
 <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[5, 6],
       [7, 8]])>


In [ ]:
# Broadcasting
a = tf.constant([[1], [2]])
b = tf.constant([3, 4])

print("\n")

print(a)

print("\n")

print(b)

print("\n")

broadcasted = a + b
print("Broadcasted Tensor:\n", broadcasted)

#   [[1]   +     [3,4] ==>   [[1+3, 1+4],
#   [2]]               ==>    [2+3, 2+4]]



tf.Tensor(
[[1]
 [2]], shape=(2, 1), dtype=int32)


tf.Tensor([3 4], shape=(2,), dtype=int32)


Broadcasted Tensor:
 tf.Tensor(
[[4 5]
 [5 6]], shape=(2, 2), dtype=int32)


In [12]:
# Reshaping
reshaped = tf.reshape(broadcasted, [4])
print("Reshaped Tensor:", reshaped)

Reshaped Tensor: tf.Tensor([4 5 5 6], shape=(4,), dtype=int32)


In [13]:
# Slicing
sliced = broadcasted[:, 1]
print("Sliced Tensor:", sliced)

Sliced Tensor: tf.Tensor([5 6], shape=(2,), dtype=int32)


🔹 2. Eager Execution vs Graph Execution


In [ ]:
#   TensorFlow supports:

#   Eager (default): operations run immediately.

#   Graph: used with @tf.function for performance.

In [14]:
@tf.function  # Compiles into graph
def compute_square(x):
    return x ** 2 + 2 * x + 1

x = tf.constant(3.0)
print("Result with tf.function (Graph):", compute_square(x))


Result with tf.function (Graph): tf.Tensor(16.0, shape=(), dtype=float32)


🔹 3. Data Pipelines using tf.data


In [15]:
# Create dataset from a list
raw_data = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])

In [16]:
# Transformation pipeline
processed = (
    raw_data
    .map(lambda x: x * 2)
    .shuffle(buffer_size=3)
    .batch(2)
    .prefetch(tf.data.AUTOTUNE)
)

for batch in processed:
    print("Processed Batch:", batch.numpy())

Processed Batch: [2 8]
Processed Batch: [ 6 10]
Processed Batch: [4]


✅ Explanation:

map: transforms each element.

shuffle: randomizes data.

batch: groups into batches.

prefetch: overlaps preprocessing and model execution → boosts performance.



🔹 4. Custom Training Loops


In [17]:
# Simple linear model y = wx + b
class LinearModel(tf.Module):
    def __init__(self):
        self.w = tf.Variable(2.0)
        self.b = tf.Variable(1.0)

    def __call__(self, x):
        return self.w * x + self.b

model = LinearModel()
optimizer = tf.optimizers.SGD(learning_rate=0.01)
x = tf.constant([1.0, 2.0, 3.0])
y_true = tf.constant([3.0, 5.0, 7.0])  # true y = 2x + 1

# Training loop
for step in range(100):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = tf.reduce_mean(tf.square(y_true - y_pred))
    grads = tape.gradient(loss, [model.w, model.b])
    optimizer.apply_gradients(zip(grads, [model.w, model.b]))

print("Trained w:", model.w.numpy(), "b:", model.b.numpy())


Trained w: 2.0 b: 1.0


🔹 5. Checkpointing and Saving Models


In [ ]:
# Save Keras model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])
])
model.compile(optimizer='sgd', loss='mse')

# Save entire model
model.save(".\my_model")  # SavedModel format

# Load model
loaded_model = tf.keras.models.load_model("my_model")
print("Model Loaded:", loaded_model.summary())

# Using tf.train.Checkpoint
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.save(file_prefix="./ckpt/model")


✅ Explanation:

model.save() saves both weights and architecture.

tf.train.Checkpoint is ideal for custom training loops.

Critical for resuming training, deployment, or versioning.